# Dependencies

In [39]:
import os
import music21 as m21
from music21 import converter, stream, environment, note
from music21.pitch import Accidental
import copy
import json
import tensorflow as tf
from tensorflow import keras
import numpy as np


from keras.utils import plot_model


us = environment.UserSettings()
us['musicxmlPath'] = '/Applications/MuseScore 3.app/Contents/MacOS/mscore'


# Data Preparation

Data (in kern format) download link: https://kern.humdrum.org/cgi-bin/browse?l=users/pchordia/bhatkhandve/3

In [40]:

kern_data_set = "Raag Bihag krn files"
save_dir = "Raag Bihag text files"
single_file_dataset = "single_file_dataset"
sequence_length = 64
mapping_path = "mapping.json"
acceptable_note_durations = [0.25, 0.5, 0.75, 1.0, 1.5, 2, 3, 4]

us = environment.UserSettings()
us['musicxmlPath'] = '/Applications/MuseScore 3.app/Contents/MacOS/mscore'

In [41]:

# Create a song list of all the songs parsed in music21 format
def create_m21_songlist (data_set):
    songs = []
    for dirpath, dirnames, filenames in os.walk(data_set):
        for file in filenames:
            if file[-3:] == "krn":
                file_path = os.path.join(dirpath,file)
                #print(file_path)   
                try:
                    song = converter.parse(file_path)
                    songs.append(song)
                except Exception as e:
                    print(f"Error parsing {file_path}: {e}")
    return songs

In [42]:
# The song contains "acciaccatura" notes. To avoid complications those notes were removed form the songs. 

def clean_songs(songs, acceptable_note_durations):
    cleaned_songs = []

    for song in songs:
        cleaned_stream = stream.Stream()

        for element in song.flat.notesAndRests:
            if element.duration.quarterLength in acceptable_note_durations:
                elem_copy = copy.deepcopy(element)  # deep copy the element
                cleaned_stream.append(elem_copy)

        cleaned_songs.append(cleaned_stream)

    return cleaned_songs




In [43]:

# Convert song to time series format
def time_series(song, time_step=0.25):
    time_series = []

    for element in song.flat.notesAndRests:
        if isinstance(element, m21.note.Note):
            value = element.pitch.midi  # fixed variable name from 'event' to 'element'
        elif isinstance(element, m21.note.Rest):
            value = "r"
        else:
            continue  # skip other elements like expressions or articulations

        steps = int(element.duration.quarterLength / time_step)
        for step in range(steps):
            if step == 0:
                time_series.append(value)
            else:
                time_series.append("_")

    # Convert all elements to string
    time_series = map(str, time_series)

    # Join with space
    return " ".join(time_series)

            
    

In [44]:

def create_mapping(songs,mapping_path):
    mappings = {}
    songs = songs.split()
    vocabulary = list(set(songs))
    for i, symbol in enumerate(vocabulary):
        mappings[symbol] = i
    with open(mapping_path, "w") as fp:
        json.dump(mappings,fp, indent=4)
        
        

In [45]:

def create_single_file(dataset_path, file_dataset_path, sequence_length):
    delimiter = "/ " * sequence_length
    songs = ""
    for path, _, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(path, file)
            with open(file_path,"r") as fp:

                song = fp.read().strip()

            songs = songs+song+" "+ delimiter
    songs = songs[:-1]
    with open (file_dataset_path, "w") as fp:
        fp.write(songs)
    return songs

In [46]:
def convert_songs_to_int(songs,mapping_path):
    int_songs = []

    # load mappings
    with open(mapping_path, "r") as fp:
        mappings = json.load(fp)

    # transform songs string to list
    songs = songs.split()

    # map songs to int
    for symbol in songs:
        int_songs.append(mappings[symbol])

    return int_songs

In [47]:
# Create two seperate list for original songs and cleaned songs
songs = create_m21_songlist (kern_data_set)
print(len(songs))
cleaned_songs = clean_songs(songs, acceptable_note_durations)
print(len(cleaned_songs))

38
38


In [48]:
#Write the time series format of songs into seperate text files
for i, song in enumerate(cleaned_songs):
    song_text = time_series(song)
    save_path = os.path.join(save_dir, str(i))
    with open(save_path, "w") as fp:
        fp.write(song_text)
        
#combine the text files into one textfile (single file data set)
songs_single_file = create_single_file(save_dir, single_file_dataset, sequence_length)

In [49]:
#create mappings
create_mapping(songs_single_file,mapping_path)

In [50]:
#generate training sequences
def load(file_path):
    with open(file_path, "r") as fp:
        return fp.read()
        
songs = load(single_file_dataset)
int_songs = convert_songs_to_int(songs,mapping_path)
inputs = []
targets = []
num_sequences = len(int_songs) - sequence_length
for i in range(num_sequences):
    inputs.append(int_songs[i:i+sequence_length])
    targets.append(int_songs[i+sequence_length])
vocabulary_size = len(set(int_songs))

inputs = keras.utils.to_categorical(inputs, num_classes=vocabulary_size)
targets = np.array(targets) 

print(f"The shape of input is: {inputs.shape}")
print(f"The shape of input is: {targets.shape}")
print(targets)

The shape of input is: (10040, 64, 17)
The shape of input is: (10040,)
[ 1  9  9 ... 15 15 15]


# Train the network

In [70]:
# Model configuration
OUTPUT_UNITS = vocabulary_size  # This is the size of the vocabulary
NUM_NEURONS = [256]
LOSS = "sparse_categorical_crossentropy"  # Typo fixed here
LEARNING_RATE = 0.001
EPOCHS = 50
BATCH_SIZE = 64
SAVE_MODEL_PATH = "my_model.keras"

In [64]:
# Build model
input_layer = keras.layers.Input(shape=(None, OUTPUT_UNITS))
x = keras.layers.LSTM(NUM_NEURONS[0])(input_layer)
x = keras.layers.Dropout(0.2)(x)
output_layer = keras.layers.Dense(OUTPUT_UNITS, activation="softmax")(x)
model = keras.Model(inputs=input_layer, outputs=output_layer,name="my_melody_model")

In [65]:
#compile model
model.compile(loss = LOSS,
              optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              metrics=["accuracy"])
                #model.summary()


In [66]:
# train model
model.fit(inputs,targets, epochs = EPOCHS, batch_size = BATCH_SIZE)

Epoch 1/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 30s 179ms/step - accuracy: 0.7661 - loss: 1.2164
Epoch 2/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 34s 216ms/step - accuracy: 0.8187 - loss: 0.8235
Epoch 3/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 39s 249ms/step - accuracy: 0.8305 - loss: 0.7099
Epoch 4/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 38s 243ms/step - accuracy: 0.8484 - loss: 0.5255
Epoch 5/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 38s 238ms/step - accuracy: 0.8650 - loss: 0.4341
Epoch 6/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 38s 239ms/step - accuracy: 0.8701 - loss: 0.4109
Epoch 7/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 37s 237ms/step - accuracy: 0.8737 - loss: 0.3770
Epoch 8/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 39s 224ms/step - accuracy: 0.8586 - loss: 0.5130
Epoch 9/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 29s 183ms/step - accuracy: 0.8828 - loss: 0.3727
Epoch 10/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 34s 215ms/step - accuracy: 0.8874 - loss: 0.3388
Epoch 11/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 33s 212ms/step - accuracy: 0.8920 - loss: 0.3283
Epoch 12/50
157/157

In [71]:
# save the model
model.save(SAVE_MODEL_PATH)

# Generating melody from the model

In [90]:


class MelodyGenerator:
    def __init__(self, model_path=SAVE_MODEL_PATH):
        self.model_path = model_path
        self.model = keras.models.load_model(model_path)

        # Load mapping
        with open(mapping_path, "r") as fp:
            self._mappings = json.load(fp)

        self._start_symbols = ["/"] * sequence_length

    def generate_melody(self, seed, num_steps, max_sequence_length):
        seed = seed.split()
        melody = seed.copy()

        # Add start symbols
        seed = self._start_symbols + seed

        # Map seed to integers
        seed = [self._mappings[symbol] for symbol in seed]

        for n in range(num_steps):
            #print(f"compositing note {n}")
            # Keep only the last `max_sequence_length` items
            seed_input = seed[-max_sequence_length:]

            # One-hot encode
            one_hot_seed = keras.utils.to_categorical(seed_input, num_classes=vocabulary_size)

            # Add batch dimension
            one_hot_seed = np.expand_dims(one_hot_seed, axis=0)

            # Predict next token
            probabilities = self.model.predict(one_hot_seed, verbose=0)[0]
            output_int = np.argmax(probabilities)

            # Append prediction
            seed.append(output_int)

            # Map back to symbol
            output_symbol = [k for k, v in self._mappings.items() if v == output_int][0]

            # End if end symbol
            if output_symbol == "/":
                break

            melody.append(output_symbol)

        return melody

            
            
            
        

In [91]:
mg = MelodyGenerator()
seed = "64 _ _ _ 65 _ _ _ 67 _ _ _ _ _ _ _ 71 _ _ _ _ _ _ _ 71 "
composition = mg.generate_melody(seed, 50, sequence_length)
print(composition)

['64', '_', '_', '_', '65', '_', '_', '_', '67', '_', '_', '_', '_', '_', '_', '_', '71', '_', '_', '_', '_', '_', '_', '_', '71', '_', '_', '_', '_', '_', '_', '_', '67', '_', '_', '_', '_', '_', '_', '_', '67', '_', '_', '_', '_', '_', '_', '_', '64', '_', '_', '_', '65', '_', '_', '_', '64', '_', '_', '_', '_', '_', '_', '_', '60', '_', '_', '_', '_', '_', '_', '_']


In [93]:
# Getting output in musescore
step_duration=0.25
format="midi"
file_name="composition.mid"

# create a music21 stream
stream = m21.stream.Stream()

start_symbol = None
step_counter = 1

# parse all the symbols in the melody and create note/rest objects
for i, symbol in enumerate(composition):

    # handle case in which we have a note/rest
    if symbol != "_" or i + 1 == len(composition):

        # ensure we're dealing with note/rest beyond the first one
        if start_symbol is not None:

            quarter_length_duration = step_duration * step_counter # 0.25 * 4 = 1

            # handle rest
            if start_symbol == "r":
                m21_event = m21.note.Rest(quarterLength=quarter_length_duration)

            # handle note
            else:
                m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)

            stream.append(m21_event)

            # reset the step counter
            step_counter = 1

        start_symbol = symbol

    # handle case in which we have a prolongation sign "_"
    else:
        step_counter += 1

# write the m21 stream to a midi file
stream.write(format, file_name)


# Load your MIDI file
score = converter.parse("composition.mid")

# Show in MuseScore
score.show('musicxml')
